In [1]:
import numpy as np
import gym
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from scipy.spatial import distance
from tqdm import tqdm_notebook
import time
%matplotlib inline

env = gym.make('MountainCarContinuous-v0')

# preprocessing
import sklearn.pipeline
import sklearn.preprocessing
from sklearn.kernel_approximation import RBFSampler
# Feature Preprocessing: Normalize to zero mean and unit variance
# We use a few samples from the observation space to do this
observation_examples = np.array([env.observation_space.sample() for x in range(10000)])
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(observation_examples)
# Used to convert a state to a featurizes represenation.
# We use RBF kernels with different variances to cover different parts of the space
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(observation_examples))
def featurize_state(state):
    """
    Returns the featurized representation for a state.
    """
    scaled = scaler.transform([state])
    featurized = featurizer.transform(scaled)
    return featurized[0]


class PolicyEstimator():
    def __init__(self, α = 0.01):
        self.θ = np.zeros(400, )
        self.σ = 1
        self.α = α
        
        self.v_dθ = np.zeros(400,)
        self.β = 0.9
        self.ε = 1e-10
        
    def mu_state(self, state):
        mu = self.θ @ state
        
        return mu
          
    def predict(self, state):
        mu = self.mu_state(state)
        
        action = np.random.normal(mu, self.σ, size=(1,))
        action = np.clip(action, env.action_space.low[0], env.action_space.high[0])
        return action
    
    def update(self, state, target, action):
        mu = self.mu_state(state)
        
        # update θ
        dloss = -((action - mu)/(self.σ**2)) * state * target
        # rmsprop optimisation
        self.v_dθ = self.β * self.v_dθ + (1 - self.β) * (dloss**2) 
        self.θ -= self.α * dloss/(np.sqrt(self.v_dθ) + self.ε)
        
class ValueEstimator():
    def __init__(self, α = 0.1):
        self.w = np.zeros(400, )
        self.α = α
        
        self.v_dw = np.zeros(400,)
        self.β = 0.9
        self.ε = 1e-10
               
    def predict(self, state):
        value = self.w @ state
        return value
    
    def update(self, state, target):
        value = self.predict(state)
               
        # update w
        dloss = 2*(value - target)*state
        # rmsprop optimisation
        self.v_dw = self.β * self.v_dw + (1 - self.β) * (dloss**2) 
        self.w -= self.α * dloss/(np.sqrt(self.v_dw) + self.ε)
        
policy_estimator = PolicyEstimator(α=0.001)
value_estimator = ValueEstimator(α=0.1)

"""
TRAIN
"""
num_episodes = 20
num_steps = 1000
discount_factor=0.95
for i in range(num_episodes):
    
    # set v_dw and v_dθ to 0
    state = env.reset()
    state = featurize_state(state)
    for t in tqdm_notebook(range(num_steps)):
        # take action
        action = policy_estimator.predict(state)
        # execute action
        next_state, reward, done, _ = env.step(action)
        next_state = featurize_state(next_state)
        # calculate TD target
        value_now = value_estimator.predict(state)
        value_next = value_estimator.predict(next_state)
        td_target = reward + discount_factor * value_next
        td_error = td_target - value_now
        # update the value estimator
        value_estimator.update(state, td_target)
        # update the policy estimator
        policy_estimator.update(state, td_error, action)
        if done & (t <= 900):
            break
        state = next_state
        
        
"""
TEST
"""
num_steps = 1000
# remove random exploration from policy
policy_estimator.σ = 0
state = env.reset()
state = featurize_state(state)
for t in tqdm_notebook(range(num_steps)):
    env.render()
    time.sleep(0.01)    
    
    # take action
    action = policy_estimator.predict(state)
    # execute action
    next_state, reward, done, _ = env.step(action)
    next_state = featurize_state(next_state)
    if done:
        break
    state = next_state
    
env.close()

/home/eek31/Documents/openai/gym/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
